# News API

A Python client for the [News API](https://newsapi.org/docs/).


## License

Provided under [MIT License](https://github.com/mattlisiv/newsapi-python/blob/master/LICENSE.txt) by Matt Lisivick.

```
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
```


## General

This is a Python client library for [News API V2](https://newsapi.org/).
The functions and methods for this library should mirror the
endpoints specified by the News API [documentation](https://newsapi.org/docs/endpoints).


## Installation

Installation for the package can be done via `pip`:

In [ ]:
!pip install newsapi-python
!pip install opencv-python
!pip install mediapipe
!pip install screeninfo
!pip install cvzone
!pip install matplotlib
!pip install pandas

## Usage

After installation, import the client class into your project:

In [ ]:
from newsapi import NewsApiClient

Initialize the client with your API key:

In [ ]:
import os

api = NewsApiClient(api_key=os.environ['NEWSAPI_ORG'])


### Endpoints

An instance of `NewsApiClient` has three instance methods corresponding to three News API endpoints.

#### Top Headlines

Use `.get_top_headlines()` to pull from the [`/top-headlines`](https://newsapi.org/docs/endpoints/top-headlines) endpoint:

In [ ]:
api.get_top_headlines(sources='bbc-news')

#### Everything

Use `.get_everything()` to pull from the [`/everything`](https://newsapi.org/docs/endpoints/everything) endpoint:

In [ ]:
api.get_everything(q='bitcoin')

#### Sources

Use `.get_sources()` to pull from the [`/sources`](https://newsapi.org/docs/endpoints/sources) endpoint:

In [ ]:
api.get_sources()

## For Windows users printing to _cmd_ or _powershell_

You will encounter an error if you attempt to print the .json() object to the command line. This is because the '{', '}' curly braces to be printed to the console.
This becomes especially annoying if developers wish to get 'under the hood'.

Here is the error:

```
UnicodeEncodeError: 'charmap' codec can't encode character '\u2019' in position 1444: character maps to <undefined>
```

This can be fixed by:
    - installing 'win-unicode-console'
        `py -mpip install win-unicode-console`
    - then running it while calling your python script...
        `py -mrun myPythonScript.py`

Another option is hardcoding your console to only print in utf-8. This is a bad idea, as it could ruin many other scripts and/or make errors MUCH more difficult to track.
[More information](https://stackoverflow.com/questions/5419/python-unicode-and-the-windows-console/32176732#32176732).


## Showtime - Let's get some news but using it together with OpenCV and Pandas


In [ ]:
import cv2
import pandas as pd
import numpy as np

In [ ]:
# init
api = NewsApiClient(api_key=os.environ['NEWSAPI_ORG'])

In [ ]:
def get_top_headlines():
    response = api.get_top_headlines(
        sources='bbc-news, cnn, fox-news, google-news, the-new-york-times, the-wall-street-journal, the-washington-post, time, usa-today, wired'
    )
    return response


top_headlines = get_top_headlines()

In [ ]:
print(
    """
    Top Headlines
    -------------
    {}
    """.format(top_headlines)
)

In [ ]:
def news_to_df(news):
    # map all the articles to a list
    articles = list(map(lambda x: x['title'], news['articles']))

    # create a dataframe
    df = pd.DataFrame(articles, columns=['title'])

    # add a column for the length of the title
    df['title_length'] = df['title'].apply(lambda x: len(x))

    # add a column for the number of words in the title
    df['title_words'] = df['title'].apply(lambda x: len(x.split(' ')))

    return df

In [ ]:
df = news_to_df(top_headlines)

In [ ]:
display(df)

In [ ]:
import cv2
import mediapipe as mp
import cvzone
import numpy as np
import screeninfo


In [15]:
# Using OpenCV to display the image
cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1024)
cap.set(cv2.CAP_PROP_FPS, 60)

# text as one line string
text = ' '.join(df['title'].tolist())

# add '###' between each title
text = ' ... | '.join(df['title'].tolist())

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

with mp_hands.Hands(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        image = cv2.flip(image, 1)

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())

                # if left hand is raised then move the text to the left
                if hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x < 0.2:
                    text = text[1:] + text[0]
                    prev_color = (255, 255, 255)
                    cv2.putText(image, text, (0, int(image.shape[0] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 1, prev_color, 2, cv2.LINE_AA)
                # if right hand is raised then move the text to the right
                if hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x > 0.8:
                    text = text[-1] + text[:-1]
                    prev_color = (255, 255, 255)
                    cv2.putText(image, text, (0, int(image.shape[0] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 1, prev_color, 2, cv2.LINE_AA)

                # if both hands are raised then change the color of the text
                if 0.2 < hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x < 0.8:
                    prev_color = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))
                    text = text
                    cv2.putText(image, text, (0, int(image.shape[0] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 1, prev_color, 4, cv2.LINE_AA)


        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('AI News', image)

        # wait for the 'q' key to be pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # if 's' is pressed, save the image
        if cv2.waitKey(1) & 0xFF == ord('s'):
            cv2.imwrite('news.jpg', image)

cap.release()

# stop the video
cv2.destroyAllWindows()


In [6]:
# Using Matplotlib to display the image

import matplotlib.pyplot as plt

# get the image
img = cv2.imread('news.jpg')

# show the image
plt.imshow(img)


NameError: name 'df' is not defined